In [30]:
import numba as nb
import numpy as np

In [45]:
#@nb.guvectorize(["void(complex128[:], UniTuple(int64, 2), complex128[:])"], "(n),(m)->()")
@nb.njit("complex128(complex128[:], uint16, uint16)")
def fast_trace(flat_array, bond_dimension, arr_shape_len):
    # Reshape flat array and compute trace.
    c_flatarray = np.ascontiguousarray(flat_array)
    arr = c_flatarray.reshape([bond_dimension for __ in range(2 * arr_shape_len)])
    trace = np.trace(arr)
    return trace

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Invalid use of BoundFunction(array.reshape for array(complex128, 1d, C)) with parameters (list(uint16)<iv=None>)

During: resolving callee type: BoundFunction(array.reshape for array(complex128, 1d, C))
During: typing of call at <ipython-input-45-5712f0c64940> (6)


File "<ipython-input-45-5712f0c64940>", line 6:
def fast_trace(flat_array, bond_dimension, arr_shape_len):
    <source elided>
    c_flatarray = np.ascontiguousarray(flat_array)
    arr = c_flatarray.reshape([bond_dimension for __ in range(2 * arr_shape_len)])
    ^


In [42]:
M = np.reshape(np.random.rand(1,10), [5,2])

In [43]:
@nb.njit
def sum_trace(arrays):
    trace = 0
    for i in range(arrays.shape[0]):
        trace += fast_trace(np.ravel(arrays[i]))
    return trace

In [44]:
sum_trace(M)

TypingError: Failed in nopython mode pipeline (step: nopython frontend)
Internal error at <numba.core.typeinfer.CallConstraint object at 0x7f807522cc18>.
missing a required argument: 'array_shape'
During: resolving callee type: type(CPUDispatcher(<function fast_trace at 0x7f8075923d08>))
During: typing of call at <ipython-input-43-7e57dd25358e> (5)

Enable logging at debug level for details.

File "<ipython-input-43-7e57dd25358e>", line 5:
def sum_trace(arrays):
    <source elided>
    for i in range(arrays.shape[0]):
        trace += fast_trace(np.ravel(arrays[i]))
        ^


In [21]:
@nb.guvectorize(["void(complex128[:,:,:], uint16, uint16, complex128[:])"],"(k,n,n),(),()->()")
def f(a, i, j, b):
    b[0] = a[i, j].sum()
    # or b[0] = a.sum()

In [25]:
@nb.njit("complex128(complex128[:,:,:], uint16, uint16)")
def f(a, i, j):
    res = a[i, j].sum()
    return res

In [26]:
a = np.random.rand(10,4,4).astype(np.complex128)
allsum = 0
for k in range(a.shape[0]):
    for i in range(a.shape[1]):
        allsum += f(a, k, i)

In [27]:
allsum

(77.66306865799434+0j)